# Initialising LUSID

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd

import helper_functions as fun 

# Authenticate our user and create our API client
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.3530.0


![Scopes](img/paper-lusid.gif)

## 1) Creating a logically separated environment for each analyst

In [2]:
# Fetch our scopes
analyst_scope_code = fun.create_analyst_scope()


Analyst Scope Code: analyst-paper-37e7-6f38-d081-a8


*Our LUSID environment now looks like the below. We will update this with each step.*

![Scopes](img/paper-scopes.gif)

## 2) Creating our tradeable instrument universe for each analyst

In [3]:
# Import our instrument universe from a CSV file using Pandas
instrument_universe = pd.read_csv('data/paper-instruments.csv')
# Look at the first 10 instruments
instrument_universe.sort_values(by="instrument_name").head(n=10)

instrument_name currency          figi ticker          isin    sedol  \
0       AVEVA GRP      GBP  BBG000C21Y87    AVV  GB00BBG9VN75  BBG9VN7   
1        BAE SYS.      GBP  BBG000BD5TW4    BA.  GB0002634946   263494   
2   BATM ADVANCED      GBP  BBG000BFJD77    BVC  IL0010849045   911146   
3             BTG      GBP  BBG000BZZK79    BTG  GB0001001592   100159   
4          COBHAM      GBP  BBG000BS6810    COB  GB00B07KD360  B07KD36   
5   COMPUTACENTER      GBP  BBG000BN7CL9    CCC  GB00BV9FP302  BV9FP30   
6    CONSORT MED.      GBP  BBG000BD8278   CSRT  GB0000946276    94627   
7        DIALIGHT      GBP  BBG000BLQNG4    DIA  GB0033057794  3305779   
8           GENUS      GBP  BBG000CTMKX7    GNS  GB0002074580   207458   
9    GRESHAM TECH      GBP  BBG000BS2592    GHT  GB0008808825   880882   

  client_internal  
0    imd_56344535  
1    imd_56344983  
2    imd_56344098  
3    imd_56344345  
4    imd_56344363  
5    imd_56344296  
6    imd_56344187  
7    imd_56344572  
8    imd_56344789  
9    imd_56344132

In [4]:
batch_upsert_request = fun.batch_upsert(
    instrument_universe, 
    api_factory)   

Instruments Successfully Upserted: 


In [5]:
# Call LUSID to upsert our batch
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    instruments=batch_upsert_request)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier='Figi').sort_values(by="Instrument").head(10)

Instruments Successfully Upserted: 


Instrument       Figi ID LUSID Instrument ID
14      AVEVA GRP  BBG000C21Y87       LUID_C63M3W9L
8        BAE SYS.  BBG000BD5TW4       LUID_PZY35CYB
0   BATM ADVANCED  BBG000BFJD77       LUID_RN4DVOPG
16            BTG  BBG000BZZK79       LUID_9G72A4LZ
7          COBHAM  BBG000BS6810       LUID_4S3J1BCC
10  COMPUTACENTER  BBG000BN7CL9       LUID_0ZN4YO4U
13   CONSORT MED.  BBG000BD8278       LUID_JCNUA8A4
11       DIALIGHT  BBG000BLQNG4       LUID_3G6G53QI
2           GENUS  BBG000CTMKX7       LUID_NM4VWVZG
26   GRESHAM TECH  BBG000BS2592       LUID_TCDOZL9W

Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/paper-instrumentmaster.gif)

## 3) Creating a transactions portfolio

A transactions portfolio is initalised to hold all transactions made by an analyst. 

A second portfolio is also initalised which will store the index of the instrument universe to be used as a benchmark.

In [6]:
# Define unique codes for each of our portfolios
transaction_portfolio_code = 'LSE_FTSE_techMarkFocus_transactions_DEMO'
reference_portfolio_code = 'LSE_FTSE_techMarkFocus_reference_DEMO'
# Pretty print our codes
prettyprint.heading('Transaction Portfolio Code', transaction_portfolio_code)
prettyprint.heading('Reference Portfolio Code', reference_portfolio_code)

# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

descriptions = "Paper transaction portfolio_DEMO"
portfolio_codes = [transaction_portfolio_code, reference_portfolio_code]


fun.request_transaction_portfolio_creation(
    transaction_portfolio_code, 
    portfolio_creation_date, 
    analyst_scope_code, 
    descriptions,
    api_factory)

portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

fun.request_reference_portfolio_creation(
    reference_portfolio_code, 
    portfolio_creation_date, 
    analyst_scope_code, 
    api_factory)

Transaction Portfolio Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Reference Portfolio Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Created
Scope: analyst-paper-37e7-6f38-d081-a8
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Portfolio Effective From: 2019-11-09 11:41:02.167178+00:00
Portfolio Created On: 2019-11-12 11:41:02.874407+00:00

Portfolio Created
Scope: analyst-paper-37e7-6f38-d081-a8
Code: LSE_FTSE_techMarkFocus_reference_DEMO
Portfolio Effective From: 2019-11-09 11:41:02.792840+00:00
Portfolio Created On: 2019-11-12 11:41:03.101303+00:00



![Scopes](img/paper-referenceportfolio.gif)

## 4) Populating our analyst's portfolio with a starting cash balance

In [7]:
# Set the date from which the cash balance will apply to be just after portfolio creation
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2.9))
# Define our initial cash balance
initial_cash_balance = 30000000

# # Create a holding adjustment to set our initial cash balance
fun.populate_with_cash(
    holdings_effective_date, 
    initial_cash_balance, 
    analyst_scope_code, 
    transaction_portfolio_code, 
    api_factory)


Holdings Successfully Set for Portfolio
Scope: analyst-paper-37e7-6f38-d081-a8
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Holdings Effective From: 2019-11-09 11:41:02.167178+00:00
Holdings Created On: 2019-11-12 11:41:02.874407+00:00



Our transactions portfolio is now populated with an initial cash balance and is no longer empty.

![Scopes](img/paper-transactionsportfoliofilled.gif)

## 5) Populating our reference portfolio 

The initaised reference portfolio is populated with correct weights such that it tracks the market as closely as possible.

In [8]:
# Import the market capitalisation of each of our instruments in the index
instrument_market_cap = pd.read_csv('data/paper-weights.csv')
# Take a look at the first 10 market capitalisations
instrument_market_cap.sort_values(by="instrument_name").head(n=10)

instrument_name currency          figi ticker          isin    sedol  \
0       AVEVA GRP      GBP  BBG000C21Y87    AVV  GB00BBG9VN75  BBG9VN7   
1        BAE SYS.      GBP  BBG000BD5TW4    BA.  GB0002634946  0263494   
2   BATM ADVANCED      GBP  BBG000BFJD77    BVC  IL0010849045   911146   
3             BTG      GBP  BBG000BZZK79    BTG  GB0001001592   100159   
4          COBHAM      GBP  BBG000BS6810    COB  GB00B07KD360  B07KD36   
5   COMPUTACENTER      GBP  BBG000BN7CL9    CCC  GB00BV9FP302  BV9FP30   
6    CONSORT MED.      GBP  BBG000BD8278   CSRT  GB0000946276  0094627   
7        DIALIGHT      GBP  BBG000BLQNG4    DIA  GB0033057794  3305779   
8           GENUS      GBP  BBG000CTMKX7    GNS  GB0002074580  0207458   
9    GRESHAM TECH      GBP  BBG000BS2592    GHT  GB0008808825  0880882   

   marketcap  
0    4502.68  
1   16349.66  
2     185.25  
3    3202.20  
4    2579.90  
5    1161.96  
6     422.56  
7     133.39  
8    1455.72  
9      64.34

In [9]:
# upsert contituents to LUSID
fun.upsert_constituents(
    instrument_market_cap, 
    holdings_effective_date, 
    analyst_scope_code, 
    reference_portfolio_code, 
    api_factory)

Constituents Upserted


![Scopes](img/paper-referenceportfoliofilled.gif)

## 6) Allow our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

In [10]:
# Import our analyst's trades from a CSV file
analyst_transactions = import_data.fetch_client_transactions(
    'paper-transactions.csv',
    days_back=2)
analyst_transactions.sort_values(by="transaction_id").head(n=10)


transaction_id       type                            portfolio  \
0   tid_124398219481        Buy  LSE_FTSE_techMarkFocus_transactions   
4   tid_223239424244   FundsOut  LSE_FTSE_techMarkFocus_transactions   
9   tid_239394829484        Buy  LSE_FTSE_techMarkFocus_transactions   
17  tid_324599699055   FundsOut  LSE_FTSE_techMarkFocus_transactions   
15  tid_334250655566        Buy  LSE_FTSE_techMarkFocus_transactions   
1   tid_339423984894   FundsOut  LSE_FTSE_techMarkFocus_transactions   
2   tid_339423984896   FundsOut  LSE_FTSE_techMarkFocus_transactions   
14  tid_342398598555   FundsOut  LSE_FTSE_techMarkFocus_transactions   
6   tid_342398989895  SellShort  LSE_FTSE_techMarkFocus_transactions   
20  tid_342453598349   FundsOut  LSE_FTSE_techMarkFocus_transactions   

   instrument_name instrument_uid                  transaction_date  \
0        AVEVA GRP   BBG000C21Y87  2019-11-11T13:36:23.591206+00:00   
4         GBP_Cash            GBP  2019-11-11T09:55:19.049340+00:00   
9         DIALIGHT   BBG000BLQNG4  2019-11-10T11:12:14.262234+00:00   
17        GBP_Cash            GBP  2019-11-10T10:37:22.054783+00:00   
15        MICROGEN   BBG000BDSS88  2019-11-10T14:06:45.378861+00:00   
1         GBP_Cash            GBP  2019-11-11T15:49:39.178658+00:00   
2         GBP_Cash            GBP  2019-11-11T14:46:33.895205+00:00   
14        GBP_Cash            GBP  2019-11-10T08:38:42.094296+00:00   
6    COMPUTACENTER   BBG000BN7CL9  2019-11-10T14:19:19.779369+00:00   
20        GBP_Cash            GBP  2019-11-11T08:34:43.917808+00:00   

                     settlement_date    units  transaction_price  \
0   2019-11-13T13:36:23.591206+00:00   265600              28.94   
4   2019-11-13T09:55:19.049340+00:00   750000               0.00   
9   2019-11-12T11:12:14.262234+00:00  1500000               4.18   
17  2019-11-12T10:37:22.054783+00:00   234000               0.00   
15  2019-11-12T14:06:45.378861+00:00   234000               3.92   
1   2019-11-13T15:49:39.178658+00:00   265600               0.01   
2   2019-11-13T14:46:33.895205+00:00   265600               0.01   
14  2019-11-12T08:38:42.094296+00:00   345000               0.01   
6   2019-11-12T14:19:19.779369+00:00   150000              10.50   
20  2019-11-13T08:34:43.917808+00:00   543000               0.01   

   transaction_currency  total_cost             strategy     description  
0                   GBP  7686464.00   quantitativeSignal        Purchase  
4                   GBP       69.45  fundamentalAnalysis  Brokerage Fees  
9                   GBP  6270000.00   quantitativeSignal        Purchase  
17                  GBP      458.64   quantitativeSignal      Stamp Duty  
15                  GBP   917280.00   quantitativeSignal        Purchase  
1                   GBP     1537.29   quantitativeSignal  Brokerage Fees  
2                   GBP     3843.23   quantitativeSignal      Stamp Duty  
14                  GBP     3795.00    technicalAnalysis      Stamp Duty  
6                   GBP  1575000.00  fundamentalAnalysis        Purchase  
20                  GBP     3692.40    technicalAnalysis      Stamp Duty

Custom properties can be defined in LUSID. In this example, a trading strategy is specified created as a **property**, this allows grouping of transactions for the purpose of measuring performance, valuation etc...

In [11]:
# Create a request to define our strategy property
strategy_property_key = fun.request_define_property(
    domain='Transaction', 
    scope =analyst_scope_code, 
    code='strategy', 
    display_name='strategy', 
    api_factory=api_factory)

Strategy Property Key: : Transaction/analyst-paper-37e7-6f38-d081-a8/strategy


In [12]:
# upsert all analyist transactions to LUSID
fun.upsert_trades(
    analyst_transactions, 
    strategy_property_key, 
    analyst_scope_code, 
    transaction_portfolio_code, 
    api_factory)

Transactions Successfully Upserted into Portfolio
Scope: analyst-paper-37e7-6f38-d081-a8
Code: LSE_FTSE_techMarkFocus_transactions_DEMO
Transactions Effective From: 2019-11-11 15:49:39.178658+00:00
Transactions Created On: 2019-11-12 11:41:04.737384+00:00



Our transaction portfolio is now populated with our analyst's paper trades.

## 7) Upload market data quotes to hold the latest prices of our tradeable instrument universe

In [13]:
# Import our instrument prices from a CSV file
instrument_prices = pd.read_csv('data/paper-analytics.csv')
# Pretty print our pricing
instrument_prices.sort_values(by="instrument_name").head(n=10)

instrument_name currency          figi  price_original  price_current  \
0       AVEVA GRP      GBP  BBG000C21Y87           27.84          30.32   
1        BAE SYS.      GBP  BBG000BD5TW4            5.15           4.78   
2   BATM ADVANCED      GBP  BBG000BFJD77            0.46           0.42   
3             BTG      GBP  BBG000BZZK79            8.28           8.97   
4          COBHAM      GBP  BBG000BS6810            1.07           1.02   
5   COMPUTACENTER      GBP  BBG000BN7CL9           10.68          11.46   
6    CONSORT MED.      GBP  BBG000BD8278            8.70           9.31   
7        DIALIGHT      GBP  BBG000BLQNG4            3.92           3.55   
8           GENUS      GBP  BBG000CTMKX7           22.46          20.60   
9    GRESHAM TECH      GBP  BBG000BS2592            0.94           1.00   

  ticker          isin    sedol  
0    AVV  GB00BBG9VN75  BBG9VN7  
1    BA.  GB0002634946   263494  
2    BVC  IL0010849045   911146  
3    BTG  GB0001001592   100159  
4    COB  GB00B07KD360  B07KD36  
5    CCC  GB00BV9FP302  BV9FP30  
6   CSRT  GB0000946276    94627  
7    DIA  GB0033057794  3305779  
8    GNS  GB0002074580   207458  
9    GHT  GB0008808825   880882

In [14]:
# Set our quote effective dates
quotes_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)
today = datetime.now(pytz.UTC)

fun.create_instrument_quotes(
    quotes_effective_date, 
    today,  
    instrument_prices, 
    analyst_scope_code, 
    api_factory)
    

![Scopes](img/paper-quotestore.gif)

## 8) Value each portfolio using our market data quotes

In [15]:
# create aggregation request
aggregation_request = fun.create_aggregation_request(analyst_scope_code, today)

# Call LUSID to aggregate across all of our portfolios
aggregated_portfolio = api_factory.build(lusid.api.AggregationApi).get_aggregation_by_portfolio(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    request=aggregation_request)

aggregated_portfolio_df = pd.DataFrame(aggregated_portfolio.data)
aggregated_portfolio_df.head(n=10)

Holding/default/SubHoldingKey  Sum(Holding/default/Units)  \
0                         Currency=GBP                  4364776.00   
1  LusidInstrumentId=LUID_KUUB54O9/GBP                   -98000.00   
2  LusidInstrumentId=LUID_05MDQMS5/GBP                   120000.00   
3  LusidInstrumentId=LUID_3G6G53QI/GBP                  1500000.00   
4  LusidInstrumentId=LUID_NM4VWVZG/GBP                   345000.00   
5  LusidInstrumentId=LUID_QQ3OG8O2/GBP                   234000.00   
6  LusidInstrumentId=LUID_0ZN4YO4U/GBP                  -150000.00   
7  LusidInstrumentId=LUID_RN4DVOPG/GBP                   750000.00   
8  LusidInstrumentId=LUID_XZ8VE9ED/GBP                    55000.00   
9  LusidInstrumentId=LUID_ZAADZHD2/GBP                  -543000.00   

   Sum(Holding/default/Cost)  Sum(Holding/default/PV)  \
0                 4364776.00               4364776.00   
1                 -577220.00               -643432.72   
2                 1582800.00               1603546.80   
3                 6270000.00               5327280.00   
4                 7590000.00               7105557.90   
5                  917280.00                843094.98   
6                -1575000.00              -1718946.00   
7                  347250.00                318431.25   
8                  305250.00                308154.00   
9                -7384800.00              -7503461.79   

   Sum(Holding/default/Price)  
0                       13.00  
1                        6.57  
2                       13.36  
3                        3.55  
4                       20.60  
5                        3.60  
6                       11.46  
7                        0.42  
8                        5.60  
9                       13.82

## 9) Compare performance across the analyst's reference and transaction portfolios

In [16]:
# create index portfolio
index_portfolio_code = 'LSE_FTSE_techMarkFocus_index_DEMO'
prettyprint.heading('Index Portfolio Code', index_portfolio_code)

description = 'Index transaction portfolio_DEMO'
fun.request_transaction_portfolio_creation(
    index_portfolio_code, 
    portfolio_creation_date, 
    analyst_scope_code, 
    description,
    api_factory)


Index Portfolio Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Created
Scope: analyst-paper-37e7-6f38-d081-a8
Code: LSE_FTSE_techMarkFocus_index_DEMO
Portfolio Effective From: 2019-11-09 11:41:02.792840+00:00
Portfolio Created On: 2019-11-12 11:41:10.756004+00:00



We now have an index portfolio that we can use to hold our index and obtain our benchmark.

![Scopes](img/paper-indexportfolio.gif)

In [17]:
# setup an index portfolio
index_setup = fun.setup_index(analyst_scope_code, reference_portfolio_code, instrument_prices, api_factory)

# Call LUSID to set all of our holdings to the initial index level
adjust_holdings_response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    effective_at=portfolio_creation_date,
    holding_adjustments=index_setup)

prettyprint.set_holdings_response(
    adjust_holdings_response, 
    analyst_scope_code, 
    index_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-37e7-6f38-d081-a8
Code: LSE_FTSE_techMarkFocus_index_DEMO
Holdings Effective From: 2019-11-09 11:41:02.792840+00:00
Holdings Created On: 2019-11-12 11:41:10.756004+00:00



We now have an index portfolio that we can value.

![Scopes](img/paper-indexportfoliofilled.gif)

In [18]:
fun.run_aggregation(analyst_scope_code, index_portfolio_code, today, api_factory)

Initial Index Level :999.98
Current Index Level :995.97
Return :-0.4013%
